<img src="../../thu_sigs_logo.png" alt="清华深研院-横" style="zoom:50%;" />

In [1]:
#| default_exp svm.use_lib

In [ ]:
#|hide
# autoreload
%load_ext autoreload
%autoreload 2
from thu_big_data_ml.help import plt

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

本文档是上文绪论的后续

In [ ]:
from thu_big_data_ml.svm.infra import compute_classification_metrics, process_sklearn_dataset_dict
from thu_big_data_ml.svm.infra import *

NameError: name 'X_train' is not defined

## 实验内容

### 调库实现SVM

为了给我们后面的实验一个参照，我们调用现有代码库的SVM，关注其精度与速度的情况。
当然如果我们Project在此收尾，只能酌情被扣除分数。
在本节之后，我们将使用 PyTorch 和 numpy 这样的基础科学计算库，来在GPU和CPU上实现SVM及其优化。

::: {.callout-important}
本次Project首先展示了几个常用的SVM库的精度与速度，并且对其进行调参；随后本次Project基于基础科学计算库手写实现了SVM及其优化，和前面的库的精度与速度进行了对比。
:::


#### sklearn实现的 SMO 形式的 RBF kernel SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train, y_train)

SVC(probability=True)

In [ ]:
y_pred = svc.predict(X_test)
y_pred_proba = svc.predict_proba(X_test)
svc_res = compute_classification_metrics(y_test, y_pred_proba, logits_to_prob=False, y_pred=y_pred, labels=list(range(10)))
svc_res


{
    'acc1': 0.9722222222222222,
    'acc2': 0.9833333333333333,
    'acc3': 0.9916666666666667,
    'acc5': 0.9916666666666667,
    'acc10': 1.0,
    'acc20': 1.0,
    'matthews_corrcoef': 0.9692503039897137,
    'f1': 0.972029880729665,
    'precision': 0.973182392919235,
    'recall': 0.971984126984127,
    'balanced_accuracy': 0.971984126984127,
    'cohen_kappa': 0.9691339500827382,
    'roc_auc': 0.9993315719260163,
    'hinge_loss': 0.11929172192787212,
    'log_loss': 0.12891302091039353,
    'acc1_pred': 0.9722222222222222
}

需要注意的是，SVM的概率计算的机制较为特殊，所以用SVM的概率预测值进行决策的话，与SVM本身的预测是有所不同的。
因此我们上面看到的结果中，根据`y_pred_proba`计算的`acc1`(top k 准确率) 与 根据 `y_pred`计算的`acc1`并不相同，SVM本身的预测比SVM的概率预测更加准确。

为了更好理解，我们代码可以看到具体的区别在哪里：

In [ ]:
interested = np.where(np.argmax(y_pred_proba, axis=1)!=y_pred)
y_pred_proba[interested], np.argmax(y_pred_proba, axis=1)[interested], y_pred[interested]


(
    array([[0.04761107, 0.06807546, 0.01568633, 0.01355811, 0.04972074,
        0.08948638, 0.29731847, 0.02387215, 0.37904699, 0.0156243 ],
       [0.00649421, 0.04747509, 0.01344191, 0.0502836 , 0.01457853,
        0.03642956, 0.0042157 , 0.39381368, 0.01326975, 0.41999797],
       [0.14115162, 0.01655808, 0.16863098, 0.17318131, 0.05128471,
        0.13025171, 0.13755381, 0.02513619, 0.03689582, 0.11935578]]),
    array([8, 9, 3]),
    array([6, 7, 2])
)

#### sklearn 实现的 SMO形式 的 linear SVM

In [ ]:
svc_linear = SVC(kernel='linear', probability=True)
svc_linear.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [ ]:
y_pred = svc_linear.predict(X_test)
y_pred_proba = svc_linear.predict_proba(X_test)
compute_classification_metrics(y_test, y_pred_proba, logits_to_prob=False, y_pred=y_pred, labels=list(range(10)))


{
    'acc1': 0.9805555555555555,
    'acc2': 1.0,
    'acc3': 1.0,
    'acc5': 1.0,
    'acc10': 1.0,
    'acc20': 1.0,
    'matthews_corrcoef': 0.9723041620958817,
    'f1': 0.9748756231876998,
    'precision': 0.9756801209103841,
    'recall': 0.9748326898326898,
    'balanced_accuracy': 0.9748326898326898,
    'cohen_kappa': 0.9722207932506817,
    'roc_auc': 0.9994876354783763,
    'hinge_loss': 0.13583030424739612,
    'log_loss': 0.12297098616968244,
    'acc1_pred': 0.975
}

#### sklearn 实现的 基于 liblinear  库 的 Linear SVM

The main differences between LinearSVC and SVC lie in the loss function used by default, and in the handling of intercept regularization between those two implementations.

In [ ]:
n_samples, n_features = X_train.shape
sklearn_will_use_dual_problem = n_samples < n_features
n_samples, n_features, sklearn_will_use_dual_problem

(1293, 64, False)

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
linear_svc = LinearSVC(penalty='l2', loss='squared_hinge', dual=sklearn_will_use_dual_problem, 
                       tol=0.0001, C=1.0, multi_class='ovr', fit_intercept=True, intercept_scaling=1)
linear_svc.fit(X_train, y_train)

LinearSVC(dual=False)

In [ ]:
y_pred = linear_svc.predict(X_test)
compute_classification_metrics(y_test, y_pred=y_pred, labels=list(range(10)), y_pred_metrics_only=True)


{
    'matthews_corrcoef': 0.9507782171913305,
    'f1': 0.9551493507947526,
    'precision': 0.9564648995608748,
    'recall': 0.9553131703131704,
    'balanced_accuracy': 0.9553131703131704,
    'cohen_kappa': 0.9506151669738929,
    'acc1_pred': 0.9555555555555556
}

#### sklearn 实现的 SGD优化Hinge Loss形式 的 linear SVM

https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.SGDClassifier.html

sklearn实现的 soft-margin linear Support Vector Machine 在 `from sklearn.linear_model import SGDClassifier`类中给出。
sklearn使用的是 one versus all (OVA) scheme，文档摘录如下
> For each of the classes, a binary classifier is learned that discriminates between that and all other classes. At testing time, we compute the confidence score (i.e. the signed distances to the hyperplane) for each classifier and choose the class with the highest confidence. 

sklearn文档指出这种形式的SVM的潜在优化，包括以下几点
- modified_huber，这是一个平滑版本的hinge loss，有研究表明效果更好。
- ASGD，这个似乎类似于深度学习中的 [Stochastic Weight Averaging (SWA)](https://pytorch.org/blog/stochastic-weight-averaging-in-pytorch/)、[Model Exponential moving average (Model EMA)](https://timm.fast.ai/training_modelEMA)和[Sharpness-Aware Minimization (SAM)](https://github.com/davda54/sam)，我们有空深入研究一下这里面的区别与联系。
- Elastic Net, 结合了L1和L2的正则项，来控制模型复杂度。
- optimal learning rate, 具体来说是 Leon Bottou 提出的一种方法，公式在[此处](https://stackoverflow.com/questions/52577305/the-formula-for-the-optimal-learning-rate-in-the-sgdclassifier-in-scikit-learn)可见。

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd_svm = SGDClassifier(
    # loss="hinge", 
# penalty="l2", 
    loss="modified_huber", 
                    penalty="elasticnet", 
                    max_iter=1000, fit_intercept=True, 
                    learning_rate='optimal', 
                    early_stopping=True, validation_fraction=0.1, n_iter_no_change=5, warm_start=True, average=True)
sgd_svm.fit(X_train, y_train)

SGDClassifier(average=True, early_stopping=True, loss='modified_huber',
              penalty='elasticnet', warm_start=True)

In [ ]:
y_pred = sgd_svm.predict(X_test)
compute_classification_metrics(y_test, y_pred=y_pred, labels=list(range(10)), y_pred_metrics_only=True)


{
    'matthews_corrcoef': 0.9476333479686607,
    'f1': 0.9522945263751336,
    'precision': 0.9529722616564722,
    'recall': 0.9525353925353925,
    'balanced_accuracy': 0.9525353925353925,
    'cohen_kappa': 0.9475277151406549,
    'acc1_pred': 0.9527777777777777
}

#### 使用更加高性能的SVM库来加速训练

##### CPU上对SVM训练的加速
我们首先尝试CPU上的加速。针对英特尔CPU，Intel对sklearn库进行了加速，我们通过下面的方式启用
```bash
pip install scikit-learn-intelex
```
参考 [博客](https://stackoverflow.com/questions/17310752/can-you-capture-the-output-of-ipythons-magic-methods-timeit) 进行测速。

In [ ]:
#| eval: false
# 加速之前
original_sklearn = [0]*4
original_sklearn[0] = %timeit -o svc.fit(X_train, y_train)
original_sklearn[1] = %timeit -o  svc_linear.fit(X_train, y_train)
original_sklearn[2] = %timeit -o linear_svc.fit(X_train, y_train)
original_sklearn[3] = %timeit -o sgd_svm.fit(X_train, y_train)

200 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
75.7 ms ± 5.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
219 ms ± 5.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
51 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from sklearnex import patch_sklearn
import logging

In [ ]:
#| eval: false
patch_sklearn(verbose=False)
logging.disable(logging.INFO)
# 因为存在JIT影响，先跑一遍
svc.fit(X_train, y_train)
svc_linear.fit(X_train, y_train)
linear_svc.fit(X_train, y_train)
sgd_svm.fit(X_train, y_train)
pass

In [ ]:
#| eval: false
# 加速之后
intel_sklearn = [0]*4
intel_sklearn[0] = %timeit -o svc.fit(X_train, y_train)
intel_sklearn[1] = %timeit -o  svc_linear.fit(X_train, y_train)
intel_sklearn[2] = %timeit -o linear_svc.fit(X_train, y_train)
intel_sklearn[3] = %timeit -o sgd_svm.fit(X_train, y_train)

173 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
76.6 ms ± 5.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
223 ms ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
50 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


[英特尔宣称](https://raw.githubusercontent.com/intel/scikit-learn-intelex/master/doc/sources/_static/scikit-learn-acceleration.PNG) 对SVC的加速可以达到203.7%，然而我们并没有看到这样的结果。
我们使用假设检验来看看英特尔的sklearn是不是显著地块？这里由于结果是相关的，而且不能假设是正态分布，所以使用 `wilcoxon` signed rank 检验。

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
#| eval: false
original_sklearn_time = [t.best for t in original_sklearn]
intel_sklearn_time = [t.best for t in intel_sklearn]
res_less = wilcoxon(intel_sklearn_time, original_sklearn_time, zero_method='zsplit',alternative='less' # 实验备则假设， intel_sklearn 用时更短
                    )
if res_less.pvalue > 0.05:
    print("Null hypothesis cannot be rejected, so I have to accept that intel_sklearn is not faster than original_sklearn. ")
res_less
res_greater = wilcoxon(intel_sklearn_time, original_sklearn_time, zero_method='zsplit', alternative='greater' # 实验备则假设，intel_sklearn 用时更长
                    )
res_less, res_greater

Fri 2024-11-15 23:22:28.921075

INFO     Null hypothesis cannot be rejected, so I have to accept that intel_sklearn is not faster than  ]8;id=606437;file:///home/ye_canming/repos/novelties/cv/cls/NamableClassify/namable_classify/utils.py\utils.py]8;;\:]8;id=941520;file:///home/ye_canming/repos/novelties/cv/cls/NamableClassify/namable_classify/utils.py#77\77]8;;\
         original_sklearn.                                                                                         

(WilcoxonResult(statistic=2.0, pvalue=0.1875), WilcoxonResult(statistic=2.0, pvalue=0.875))

可以看出，不能显著地说哪个更快，都是在随机误差范围之内。可以认为这两个方法一样快。

##### GPU上对SVM的加速

```bash
pip install thundersvm
```

In [ ]:
#| eval: false
from thundersvm import SVC

In [ ]:
#| eval: false
thunder_svc = SVC(kernel='rbf', probability=True)
thunder_svc.fit(X_train, y_train)

: 

In [ ]:
import pandas as pd 

In [ ]:
#| eval: false
y_pred = thunder_svc.predict(X_test)
y_pred_proba = thunder_svc.predict_proba(X_test)
thunder_svc_res = compute_classification_metrics(y_test, y_pred_proba, logits_to_prob=False, y_pred=y_pred, labels=list(range(10)))
# thunder_svc_res, svc_res
res = []
for k, v in thunder_svc_res.items():
    delta = v - svc_res[k]
    # print(f"{k}: {v:.4f} ({delta:+.4f})")
    res.append(dict(metric=k, value=v, delta=delta))
pd.DataFrame(res)

,metric,value,delta
0,acc1,0.972222,0.000000
1,acc2,0.988889,0.005556
2,acc3,0.991667,0.002778
3,acc5,0.994444,0.002778
4,acc10,1.000000,0.000000
5,acc20,1.000000,0.000000
6,matthews_corrcoef,0.969250,0.000000
7,f1,0.972030,0.000000
8,precision,0.973182,0.000000
9,recall,0.971984,0.000000


看来精度上和sklearn的SVC基本一致，基本没有差别，除了`log_loss`多一点。

In [ ]:
#| eval: false
thunder_svc_time = %timeit -o thunder_svc.fit(X_train, y_train)

164 ms ± 59.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#| eval: false
svc_time = %timeit -o svc.fit(X_train, y_train)

186 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


这里时间是不相关的，所以使用 `mannwhitneyu` 检验。

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
#| eval: false
res_less = mannwhitneyu(thunder_svc_time.all_runs, svc_time.all_runs, alternative='less' # 实验备则假设， intel_sklearn mannwhitneyu
                    )
if res_less.pvalue > 0.05:
    print("We cannot reject the null hypothesis! ")
else:
    print("Null hypothesis rejected! ThunderSVC is significantly faster than SVC.")
res_less

Fri 2024-11-15 23:22:46.455835

INFO     Null hypothesis rejected! ThunderSVC is significantly faster than SVC.                         ]8;id=545742;file:///home/ye_canming/repos/novelties/cv/cls/NamableClassify/namable_classify/utils.py\utils.py]8;;\:]8;id=458913;file:///home/ye_canming/repos/novelties/cv/cls/NamableClassify/namable_classify/utils.py#77\77]8;;\

MannwhitneyuResult(statistic=0.0, pvalue=0.0002913752913752914)

看来`thundersvm`在GPU上加速还是显著地更快一些。

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()